In [1]:
import numpy as np
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn import datasets

# Data Preparation

In [2]:
diabetes = datasets.load_diabetes()
x = diabetes.data
y_not_standardized = diabetes.target

x is already standardized. however y is not.

In [3]:
y = y_not_standardized - y_not_standardized.mean()
print(y_not_standardized.mean())
print(y.mean())

152.13348416289594
-7.716301202824617e-15


# lars function

In [8]:
def lars(x, y, tol=1e-10):
    # initialization
    n = x.shape[0]
    m = x.shape[1]
    beta = np.zeros(m)
    prediction = np.zeros(n)
    active_set = []
    signs = np.zeros(m)
    tol=1e-8
    beta_path = [beta.copy()] 
    prediction_path = [prediction.copy()]
    active_sets = [active_set.copy()]

    for step in range(m):
        residual = y - prediction    
        correlations = x.T @ residual
        c_max = np.max(np.abs(correlations))
        if c_max < tol:
            break
        candidates = np.where(np.abs(np.abs(correlations)-c_max) <= tol)[0] # can i do this better?
        for j in candidates:
            if j not in active_set:
                active_set.append(int(j))
                signs[j] = np.sign(correlations[j]) if correlations[j] != 0.0 else 1.0 # check this
        nump_active_set = np.array(active_set, dtype=int) # consider adding dype elsewhere
        active_signs = signs[nump_active_set]
        active_x = x[:, nump_active_set] * active_signs
        g_a = active_x.T @ active_x
        g_a_inverse = np.linalg.inv(g_a)
        ones_a = np.ones(len(nump_active_set))
        a_a = 1.0 / np.sqrt(ones_a.T @ g_a_inverse @ ones_a)
        w_a = a_a * (g_a_inverse @ ones_a)
        u_a = active_x @ w_a
        little_a = x.T @ u_a
        
        #find gamma step size
        gamma = np.inf
        j_star = None
        inactive = [j for j in range(m) if j not in active_set]
        for j in inactive:
            denominator_1 = a_a - little_a[j]
            if denominator_1 > tol:
                gamma1 = (c_max - correlations[j]) / denominator_1
                if gamma1 > tol and gamma1 < gamma:
                    gamma = gamma1
                    j_star = j
            denominator_2 = a_a + little_a[j]
            if denominator_2 > tol:
                gamma2 = (c_max + correlations[j]) / denominator_2
                if gamma2 > tol and gamma2 < gamma:
                    gamma = gamma2
                    j_star = j
        
        # update prediction and beta
        prediction += gamma * u_a
        for index, j in enumerate(nump_active_set):
            beta[j] += gamma * w_a[index] * signs[j]
        if not inactive:
            break
        if j_star not in active_set: # do i need this condition?
            active_set.append(j_star)
            signs[j_star] = np.sign(correlations[j_star]) if correlations[j_star] != 0.0 else 1.0

        beta_path.append(beta.copy())
        prediction_path.append(prediction.copy())
        active_sets.append(active_set.copy())


    beta_path = np.vstack(beta_path)
    prediction_path = np.vstack(prediction_path)


    return beta_path, prediction_path, active_sets

In [9]:
beta_path, prediction_path, active_sets = lars(x, y)
print(active_sets)

[[], [2, 8], [2, 8, 3], [2, 8, 3, 6], [2, 8, 3, 6, 1], [2, 8, 3, 6, 1, 9], [2, 8, 3, 6, 1, 9, 4], [2, 8, 3, 6, 1, 9, 4, 7], [2, 8, 3, 6, 1, 9, 4, 7, 5], [2, 8, 3, 6, 1, 9, 4, 7, 5, 0]]


# Plot results

In [10]:
K_plus_1, m = beta_path.shape
n = x.shape[0]

# x-axis for left panel: L1 norm sum_j |beta_j|
l1_norm = np.sum(np.abs(beta_path), axis=1)          # (K+1,)

# residuals and correlations for right panel
residual_path = y[None, :] - prediction_path        # (K+1, n)
corr_path = residual_path @ x                       # (K+1, m)
abs_corr_path = np.abs(corr_path)                   # |c_kj|
C_k = np.max(abs_corr_path, axis=1)                 # max_j |c_kj|
steps = np.arange(K_plus_1)                         # 0..K

In [11]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("LARS (coefficients)", "LARS (correlations)")
)

# ----- Left panel: beta_j vs sum |beta_j| -----
for j in range(m):
    fig.add_trace(
        go.Scatter(
            x=l1_norm,
            y=beta_path[:, j],
            mode="lines",
            name=f"β{j+1}",
            showlegend=False  # you can turn on if you want a legend
        ),
        row=1, col=1
    )

fig.update_xaxes(title_text="∑ |β̂_j| →", row=1, col=1)
fig.update_yaxes(title_text="β̂_j", row=1, col=1)

# ----- Right panel: |c_kj| vs step k -----
for j in range(m):
    fig.add_trace(
        go.Scatter(
            x=steps,
            y=abs_corr_path[:, j],
            mode="lines",
            line=dict(width=1),
            showlegend=False
        ),
        row=1, col=2
    )

# heavy curve: C_k = max_j |c_kj|
fig.add_trace(
    go.Scatter(
        x=steps,
        y=C_k,
        mode="lines+markers",
        line=dict(width=3),
        name="Ĉ_k"
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Step k →", row=1, col=2)
fig.update_yaxes(title_text="|ĉ_kj|", row=1, col=2)

fig.update_layout(title_text="LARS analysis of diabetes data (reproduction)")
fig.show()
